In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from itertools import chain

In [6]:
baseurl = 'https://www.thewhiskyexchange.com/'

In [7]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

## Get Link of each product

In [11]:
product_links = []
# we get links from x pages
for x in range(1,3):
        r = requests.get('https://www.thewhiskyexchange.com/c/35/japanese-whisky?pg={x}')
        soup = BeautifulSoup(r.content,'lxml')
        product_list = soup.find_all('li', class_='product-grid__item')
        for item in product_list:
            for link in item.find_all('a', href = True):
                            product_links.append(baseurl+link['href'])

## Get the data : Product_name , Product_price ,  Product_rating

In [13]:
wiskylist = []
for link in product_links:
    r = requests.get(link,headers = headers)
    soup = BeautifulSoup(r.content,'lxml')
    Product_name = soup.find('h1',class_='product-main__name').text.strip()
    try:
        Product_rating = soup.find('span',class_='review-overview__rating star-rating star-rating--20').text.strip()
    except:
        Product_rating = 'No rating'
    Product_price = soup.find('p',class_='product-action__price').text
    wisky = {
              'Product_name':Product_name,
              'Product_rating':Product_rating,
              'Product_price':Product_price
             }
    wiskylist.append(wisky)


df = pd.DataFrame(wiskylist)
df

,Product_name,Product_rating,Product_price
0,Suntory Hibiki Harmony,No rating,£76.95
1,Yamazaki 12 Year Old,No rating,£150
2,Yamazaki Distiller's Reserve,No rating,£72.95
3,Ichiro's Malt Double Distilleries 2021,2,£299
4,Yamazaki 18 Year Old,No rating,£950
5,Nikka Coffey Grain Whisky,No rating,£57.95
6,Shizuoka Contact S Single Malt\n3 Year Old,No rating,£175
7,Hakushu Distiller's Reserve,No rating,£70.95
8,Yoichi Single Malt,No rating,£77.75
9,Shinju 8 Year Old Japanese Whisky,No rating,£59.75


#  Another way && Scrape more infos

### Function return the link of each product

In [14]:
def get_page_link(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    links = soup.select('li.product-grid__item a')  
    return ['https://www.thewhiskyexchange.com'+link.attrs['href'] for link in links]

### Funtion return data needed : title , price , stock , description

In [15]:
def get_data(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    product = {
        'title': soup.select_one('h1.product-main__name').text.strip().replace('\n','') ,
        'price': soup.select_one('p.product-action__price').text.strip().replace('£','')  ,
        'stock': soup.select_one('p.product-action__stock-flag').text.strip().replace('\n','') ,
        'Description': soup.select_one('div.product-main__description p').text.strip().replace('\n','') 
       
    }
    return product
    
    

### Main function execution

In [16]:
def main():
    result = []
    for x in range(1,4):
        # the url of each product in pages 1 -> 4
        urls = get_page_link(f'https://www.thewhiskyexchange.com/c/305/rest-of-the-world-whisky?pg={x}')
        
        # product infos 
        productinfo = [get_data(url) for url in urls]
        result.append(productinfo)
        print(f'page {x} completed')
    return result

# turn the data into DataFrame
df = pd.DataFrame(list(chain.from_iterable(main())))
df


page 1 completed
page 2 completed
page 3 completed


,title,price,stock,Description
0,Penderyn Sherrywood,42.95,In Stock,"A bottle of the popular Welsh whisky Penderyn,..."
1,Nikka From the Barrel,41.95,In Stock,Nikka from the Barrel is big-boned and burstin...
2,Penderyn Rich Oak,45.95,In Stock,A permanent addition to the beloved Welsh dist...
3,Indri Trini Three Wood Indian Single Malt,39.45,In Stock,"An Indian single malt from Indri, Trini has be..."
4,The Gospel Straight Rye,49.95,In Stock,An Australian straight rye whiskey from The Go...
...,...,...,...,...
67,The English Original Single Malt Whisky,44.25,In Stock,The ongoing unpeated release from The English ...
68,Stauning Smoke Single Malt,74.25,In Stock,A rich and smoky single malt from Danish whisk...
69,Bimber Apogee XII12 Year Old,59.95,In Stock,"A blend of malt whiskies sourced in Scotland, ..."
70,Langatun 2016Pedro Ximenez Sherry Cask Finish,83.95,In Stock,"A 2016 Langatun Swiss single malt whisky, fini..."


### Turn the data into excel file

In [ ]:
df.to_excel('Wisky.xlsx')